In [ ]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np 
import glob
from api import Detector
import json
import time

from utils import visualization
from utils import vid_utils as vid
from utils import repp_rotated_api as repp

In [ ]:
vid_names = ['warehouse_short']

# RAPiD-FA

In [ ]:
# RAPiD-FA
weights_path = "weights/RAPiD_FA.ckpt"
in_path = "examples"
out_path = "outputs/RAPiD_FA"
os.makedirs(out_path, exist_ok=True)
flow_warping = False
detector = Detector(model_name='rapid', weights_path=weights_path)

# Load bbox assocation model of REPP
repp_cfg = "utils/cepdof_repp_cfg.json"
repp_params = json.load(open(repp_cfg, 'r'))    
repp_model = repp.REPP(**repp_params)

vid_conf = 0.3
os.makedirs(os.path.join(out_path, "txts"), exist_ok=True)
os.makedirs(os.path.join(out_path, "jsons"), exist_ok=True)
os.makedirs(os.path.join(out_path, f"vids/conf{vid_conf}"), exist_ok=True)

for vid_name in vid_names:
    print(f"Dealing with {vid_name}")
    vid_folder = os.path.join(in_path, vid_name)
    txt_det_folder = os.path.join(out_path, "txts", vid_name)
    json_det_path = os.path.join(out_path, "jsons", f"{vid_name}_dets.json")
    
    # Compute detections
    print("Computing detections...")
    
    tic = time.perf_counter()
    vid.saveDetsFgfa(
        detector, 
        vid_folder, 
        txt_det_folder, 
        flow_warping=flow_warping,
        frame_diff=4,
        input_size=1024,
    )
    toc = time.perf_counter()
    elapsed = toc - tic
    num_frames = len(os.listdir(vid_folder))
    fps = num_frames / elapsed
    print(f"Computed detections of {num_frames} frames in {elapsed:0.4f} seconds: {fps:0.4f} FPS")

    # Convert detections from txt to json
    dets_arr = vid.txt2json(txt_det_folder, json_path=json_det_path)
    
    # REPP box association
    print("Associate BBs with REPP similarity...")
    dets_dict = repp.det_arr_to_det_dict(dets_arr, root_dir_frames=vid_folder)
    dets_repp = repp_model(dets_dict, postproc=False)
    json.dump(dets_repp, open(json_det_path, 'w'), indent=4)  
    toc = time.perf_counter()
    elapsed = toc - tic
    num_frames = len(os.listdir(vid_folder))
    fps = num_frames / elapsed
    print(f"Computed detections of {num_frames} frames in {elapsed:0.4f} seconds: {fps:0.4f} FPS")
    
    # Produce a video of detections
    print("Creating video...")
    im2dets = vid.json2dict(json_det_path, include_ids=True)
    im2dets_arr = [(im2dets, (0, 255, 0), True, vid_conf)]
    vid_path = os.path.join(out_path, f"vids/conf{vid_conf}", f"{vid_name}_dets.mp4")
    vid.saveVideo(vid_folder, im2dets_arr, vid_path, show_count=True, show_id=True)
print("DONE!")

# RAPiD-FGFA

In [ ]:
# RAPiD-FGFA
weights_path = "weights/RAPiD_FGFA.ckpt"
in_path = "examples"
out_path = "outputs/RAPiD_FGFA"
os.makedirs(out_path, exist_ok=True)
flow_warping = True
detector = Detector(model_name='rapid', weights_path=weights_path)

# Load bbox assocation model of REPP
repp_cfg = "utils/cepdof_repp_cfg.json"
repp_params = json.load(open(repp_cfg, 'r'))    
repp_model = repp.REPP(**repp_params)

vid_conf = 0.3
os.makedirs(os.path.join(out_path, "txts"), exist_ok=True)
os.makedirs(os.path.join(out_path, "jsons"), exist_ok=True)
os.makedirs(os.path.join(out_path, f"vids/conf{vid_conf}"), exist_ok=True)

for vid_name in vid_names:
    print(f"Dealing with {vid_name}")
    vid_folder = os.path.join(in_path, vid_name)
    txt_det_folder = os.path.join(out_path, "txts", vid_name)
    json_det_path = os.path.join(out_path, "jsons", f"{vid_name}_dets.json")
    
    # Compute detections
    print("Computing detections...")
    
    tic = time.perf_counter()
    vid.saveDetsFgfa(
        detector, 
        vid_folder, 
        txt_det_folder, 
        flow_warping=flow_warping,
        frame_diff=4,
        input_size=1024,
    )
    toc = time.perf_counter()
    elapsed = toc - tic
    num_frames = len(os.listdir(vid_folder))
    fps = num_frames / elapsed
    print(f"Computed detections of {num_frames} frames in {elapsed:0.4f} seconds: {fps:0.4f} FPS")

    # Convert detections from txt to json
    dets_arr = vid.txt2json(txt_det_folder, json_path=json_det_path)
    
    # REPP box association
    print("Associate BBs with REPP similarity...")
    dets_dict = repp.det_arr_to_det_dict(dets_arr, root_dir_frames=vid_folder)
    dets_repp = repp_model(dets_dict, postproc=False)
    json.dump(dets_repp, open(json_det_path, 'w'), indent=4)  
    toc = time.perf_counter()
    elapsed = toc - tic
    num_frames = len(os.listdir(vid_folder))
    fps = num_frames / elapsed
    print(f"Computed detections of {num_frames} frames in {elapsed:0.4f} seconds: {fps:0.4f} FPS") 
    
    # Produce a video of detections
    print("Creating video...")
    im2dets = vid.json2dict(json_det_path, include_ids=True)
    im2dets_arr = [(im2dets, (0, 255, 0), True, vid_conf)]
    vid_path = os.path.join(out_path, f"vids/conf{vid_conf}", f"{vid_name}_dets.mp4")
    vid.saveVideo(vid_folder, im2dets_arr, vid_path, show_count=True, show_id=True)
print("DONE!")

# RAPiD-REPP

In [ ]:
# RAPiD-REPP
weights_path = "weights/RAPiD.ckpt"
in_path = "examples"
out_path = "outputs/RAPiD_REPP"
os.makedirs(out_path, exist_ok=True)
flow_warping = False
detector = Detector(model_name='rapid', weights_path=weights_path)

# Load bbox assocation model of REPP
repp_cfg = "utils/cepdof_repp_cfg.json"
repp_params = json.load(open(repp_cfg, 'r'))    
repp_model = repp.REPP(**repp_params)

vid_conf = 0.3
os.makedirs(os.path.join(out_path, "txts"), exist_ok=True)
os.makedirs(os.path.join(out_path, "jsons"), exist_ok=True)
os.makedirs(os.path.join(out_path, f"vids/conf{vid_conf}"), exist_ok=True)

for vid_name in vid_names:
    print(f"Dealing with {vid_name}")
    vid_folder = os.path.join(in_path, vid_name)
    txt_det_folder = os.path.join(out_path, "txts", vid_name)
    json_det_path = os.path.join(out_path, "jsons", f"{vid_name}_dets.json")
    
    # Compute detections
    print("Computing detections...")
    
    tic = time.perf_counter()
    vid.saveDets(
        detector, 
        vid_folder, 
        txt_det_folder, 
    )

    # Convert detections from txt to json
    dets_arr = vid.txt2json(txt_det_folder, json_path=json_det_path)
    
    # REPP post-processing
    print("Post-processing with REPP...")
    dets_dict = repp.det_arr_to_det_dict(dets_arr, root_dir_frames=vid_folder)
    dets_repp = repp_model(dets_dict)
    json.dump(dets_repp, open(json_det_path, 'w'), indent=4)  
    toc = time.perf_counter()
    elapsed = toc - tic
    num_frames = len(os.listdir(vid_folder))
    fps = num_frames / elapsed
    print(f"Computed detections of {num_frames} frames in {elapsed:0.4f} seconds: {fps:0.4f} FPS")  
    
    # Produce a video of detections
    print("Creating video...")
    im2dets = vid.json2dict(json_det_path, include_ids=True)
    im2dets_arr = [(im2dets, (0, 255, 0), True, vid_conf)]
    vid_path = os.path.join(out_path, f"vids/conf{vid_conf}", f"{vid_name}_dets.mp4")
    vid.saveVideo(vid_folder, im2dets_arr, vid_path, show_count=True, show_id=True)
print("DONE!")